# Setup

In [1]:
# System imports
import os
import re
from dotenv import load_dotenv

# Python libraries
import requests

# Gradio imports
import gradio as gr

# OpenAI imports
from openai import OpenAI

# BeautifulSoup imports
from bs4 import BeautifulSoup

/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the environment variables
load_dotenv()

# Retrieve the API key from the environment variable
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
# Define API endpoints
list_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/posts'
search_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/articles?query='

# Fetch Posts from API

In [ ]:
# Function to fetch posts from API
def fetch_posts_from_api(page):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(list_posts_url, params={'page': page}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        # print("Response Headers:", response.headers)
        # print("Response Text:", response.text)

        post_data = response.json()

        return post_data['posts']
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

In [ ]:
# Fetch posts
page = 1
posts = fetch_posts_from_api(page=page)
print(f"Fetched {len(posts)} posts.")

# # Display the first post to verify the structure
# if posts:
#     print(posts[0])

# Search Posts from API

In [4]:
# Function to search posts from API
def search_posts(query):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(search_posts_url, params={'query': query}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        # print("Response Headers:", response.headers)
        # print("Response Text:", response.text)

        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

# Query Preprocessing

In [ ]:
# Search for posts with a keyword
query = "Apple"

In [5]:
# Fetch search results
search_results = search_posts(query)

# Display the search results
if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
    search_posts = search_results['posts']['hits']

    # Articles with title and content
    raw_articles = "".join([post['title'] + post['content'] for post in search_posts[:3]])
else:
    raw_articles = 'No relevant posts found.'

Status Code: 200
Context: Apple’s Mac event has Chinese fans thinking about buying a Surface<img src="https://cdn.techinasia.com/wp-content/uploads/2016/10/Macbook-1024x769-750x563.jpg" alt="macbook-1024x769" width="750" height="563" class="aligncenter size-large wp-image-361217" />

Apple's Mac event last week didn't look great in comparison with Microsoft's surprisingly great event the day before. On Friday I wrote that <a href="https://www.techinasia.com/microsoft-apple">Microsoft is the new Apple</a>, and it seems like China's tech community may be thinking the same thing.

Apple's upgraded MacBook Pro line obviously made headlines across all of China's tech news sites, but commenters seem decidedly underwhelmed by Apple's innovations this go-round. For example, one popular comment <a href="http://comment5.news.sina.com.cn/comment/skin/default.html?channel=kj&newsid=comos-fxxfuff6995630">on Sina Tech</a> reads:

<blockquote>Apple's launch events are getting more and more disappoint

# Articles Preprocessing

In [7]:
def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Handle special characters
    text = re.sub(r'&amp;', '&', text)

    # Normalise whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove source and citation tags
    text = re.sub(r'\[.*?\]+', '', text)

    return text

In [9]:
# Apply preprocessing
cleaned_articles = preprocess_text(raw_articles)
print(cleaned_articles)

# Length of cleaned content
print(f"Length of raw content: {len(raw_articles)}")
print(f"Length of cleaned content: {len(cleaned_articles)}")

Apple’s Mac event has Chinese fans thinking about buying a Surface Apple's Mac event last week didn't look great in comparison with Microsoft's surprisingly great event the day before. On Friday I wrote that Microsoft is the new Apple, and it seems like China's tech community may be thinking the same thing. Apple's upgraded MacBook Pro line obviously made headlines across all of China's tech news sites, but commenters seem decidedly underwhelmed by Apple's innovations this go-round. For example, one popular comment on Sina Tech reads: Apple's launch events are getting more and more disappointing. Originally I was thinking I'd wait for the upgraded MacBook Pro, but it turns out all they "upgraded" was the price. It's just not worth it. Can Apple get back to the transformative way it was in the Steve Jobs era? Now it seems its attempts at change are very stiff. Like the ports. In the Jobs era, everybody was excited by the changes Apple made. If you just keep slowly chewing through your c

# Generate Response

In [ ]:
# Function to process search results and generate background knowledge
def generate_context(search_results):
    if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
        search_posts = search_results['posts']['hits']

        # background knowledge with title and content
        raw_articles = "".join([post['title'] + post['content'] for post in search_posts[:3]])
    else:
        raw_articles = 'No relevant posts found.'
    return raw_articles

In [ ]:
# Function to generate responses using OpenAI
def generate_response(search_results, query):
    try:
        background_knowledge = generate_context(search_results)

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Answer the question based on the question asked and background knowledge provided below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Question: {query}\nBackground Knowledge: {background_knowledge}\nAnswer:"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(f'Error generating response: {e}')
        return None

In [ ]:
# # Test the response generation
# query = 'How is Grab performing in the market?'

# search_results = search_posts(query)
# generate_response(search_results, query)

# Interface

In [ ]:
# Build the Gradio interface
def rag_interface(query):
    search_results = search_posts(query)
    return generate_response(search_results, query)

iface = gr.Interface(fn=rag_interface, inputs="text", outputs="text", title='Tech in Asia RAG System', description='Ask a question about Tech in Asia and get an answer based on the context of the latest posts.')
iface.launch()